In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import numpy as np

In [ ]:
!pip install np_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56441 sha256=3feac89ffbb18cb3421193c64aa4f74fd5da16b36581a9659ab20e0dc8ee5d44
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils


In [ ]:
import np_utils

In [ ]:
import keras.utils

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
print("X-train shape: ", X_train.shape)
print("X-test shape: ", X_test.shape)
print("y-train shape: ", y_train.shape)
print("y-test shape: ", y_test.shape)

X-train shape:  (60000, 28, 28)
X-test shape:  (10000, 28, 28)
y-train shape:  (60000,)
y-test shape:  (10000,)


In [ ]:
def build_data(classes, total_classes, X_train_all, y_train_all, X_test_all, y_test_all):
  train_ind = []
  test_ind = []

  for c in classes:
    train_ind.extend(list(np.where(y_train_all == c)[0]))
    test_ind.extend(list(np.where(y_test_all == c)[0]))

  X_train = X_train_all[train_ind, :, :]
  X_test = X_test_all[test_ind, :, :]

  y_train_true = y_train_all[train_ind]
  y_train = np.zeros(y_train_true.shape)

  y_test_true = y_test_all[test_ind]
  y_test = np.zeros(y_test_true.shape)

  for i, c in enumerate(classes):
    train_ind = list(np.where(y_train_true == c)[0])
    test_ind = list(np.where(y_test_true == c)[0])
    y_train[train_ind] = i;
    y_test[test_ind] = i;

  X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
  X_train = X_train.astype('float32')/255.
  X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
  X_test = X_test.astype('float32')/255.

  # Convert class vectors to binary class matrices
  Y_train = keras.utils.to_categorical(y_train, total_classes)
  Y_test = keras.utils.to_categorical(y_test, total_classes)

  return X_train, Y_train, X_test, Y_test

In [ ]:
def build_model(old_model = None):
  model = Sequential()

  if old_model is None:
    # Add padding='same' to keep the output size the same as the input size after convolution
    model.add(Conv2D(nb_filters,
                     nb_conv,
                     padding = 'same',  # Change padding to 'same'
                     input_shape = (1, img_rows, img_cols)))
  else:
    weights = old_model.layers[0].get_weights()
    print("1.weights:{}".format(weights))

    model.add(Conv2D(nb_filters, nb_conv, nb_conv,
                     padding = 'same',
                     weights = weights, input_shape = (1, img_rows, img_cols)))
    model.add(Activation('relu'))

  if old_model is None:
    # Add padding='same' to keep the output size the same as the input size after convolution
    model.add(Conv2D(nb_filters, nb_conv, nb_conv, padding = 'same'))  # Change padding to 'same'
  else:
    # Access the correct Conv2D layer from the old model (index 1 instead of 2)
    weights = old_model.layers[2].get_weights()
    model.add(Conv2D(nb_filters, nb_conv, nb_conv, padding = 'same', weights = weights))
  # print("2.weights:{}".format(weights))
  model.add(Activation('relu'))
  # Set padding to 'same' for MaxPooling2D to avoid reducing spatial dimensions below 1.
  model.add(MaxPooling2D(pool_size = (nb_pool, nb_pool), padding='same')) # Add padding='same' here
  model.add(Dropout(0.25))

  model.add(Flatten())

  if old_model is None:
    model.add(Dense(128))
  else:
    weights = old_model.layers[7].get_weights()

    print("3.weights:{}".format(weights))
    model.add(Dense(128, weights = weights))
  #print("4.weights:{}".format(weights))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))

  return model

In [ ]:
img_rows, img_cols = 28, 28
(X_train_all, y_train_all), (X_test_all, y_test_all) = mnist.load_data()

classes = [0, 1]
nb_filters = 32
nb_pool = 2
nb_conv = 3

# Build data for the selected classes
X_train, Y_train, X_test, Y_test = build_data(classes, 2, X_train_all, y_train_all, X_test_all, y_test_all)

model1 = build_model()
model1.add(Dense(len(classes)))
model1.add(Activation('softmax'))

# Correct the loss function name
model1.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

# Fit the model using the processed Y_train (one-hot encoded labels)
model1.fit(X_train, Y_train, batch_size=256, epochs=5, verbose=1, validation_data=(X_test, Y_test))

score1 = model1.evaluate(X_test, Y_test, verbose = 0)
print('Loss score: ', score1[0])
print('Test accuracy: ', score1[1])

Epoch 1/5
50/50 [==============================] - 2s 22ms/step - loss: 0.6868 - accuracy: 0.5531 - val_loss: 0.6811 - val_accuracy: 0.7002
Epoch 2/5
50/50 [==============================] - 1s 18ms/step - loss: 0.6859 - accuracy: 0.5632 - val_loss: 0.6786 - val_accuracy: 0.7314
Epoch 3/5
50/50 [==============================] - 1s 19ms/step - loss: 0.6831 - accuracy: 0.5677 - val_loss: 0.6760 - val_accuracy: 0.7650
Epoch 4/5
50/50 [==============================] - 1s 18ms/step - loss: 0.6791 - accuracy: 0.5797 - val_loss: 0.6733 - val_accuracy: 0.7943
Epoch 5/5
50/50 [==============================] - 1s 19ms/step - loss: 0.6781 - accuracy: 0.5868 - val_loss: 0.6705 - val_accuracy: 0.8265
Loss score:  0.6705046892166138
Test accuracy:  0.8264775276184082


In [ ]:
model1.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 1, 28, 32)         8096      
                                                                 
 conv2d_15 (Conv2D)          (None, 1, 10, 32)         9248      
                                                                 
 activation_15 (Activation)  (None, 1, 10, 32)         0         
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 1, 5, 32)          0         
 g2D)                                                            
                                                                 
 dropout_8 (Dropout)         (None, 1, 5, 32)          0         
                                                                 
 flatten_4 (Flatten)         (None, 160)               0         
                                                      

In [ ]:
# Save this model for later
json_string = model1.to_json() # Call the to_json() method on the model1 object
open('model1_incremental_architecture.json', 'w').write(json_string)
model1.save_weights('model1_incremental.weights.h5')

In [ ]:
total_classes = 10
model2 = build_model(old_model=model1)
model2.add(Dense(total_classes))

# Get weights from the penultimate layer of model1 (after training)
old_weights = model1.layers[-2].get_weights()

# Get weights from the newly added Dense layer in model2
new_weights = model2.layers[-1].get_weights()

# Transfer the weights for the shared classes
new_weights[0][:, -len(classes):] = old_weights[0]
new_weights[1][-len(classes):] = old_weights[1]

# Set the updated weights back to the Dense layer in model2
model2.layers[-1].set_weights(new_weights)

1.weights:[array([[[[ 4.49189320e-02,  8.68238732e-02,  6.35428950e-02, ...,
           2.21313164e-02,  3.22128311e-02, -3.97522673e-02],
         [ 1.00685604e-01,  4.56830785e-02, -7.73530751e-02, ...,
          -3.08964178e-02,  9.66419503e-02,  2.18845531e-02],
         [-4.84520495e-02, -2.16001943e-02,  7.27337524e-02, ...,
          -6.68609440e-02, -5.76342531e-02,  6.25455603e-02],
         ...,
         [-2.65408754e-02,  1.30553022e-02, -8.69002193e-02, ...,
          -6.41026795e-02, -9.50030461e-02,  2.57402286e-02],
         [-7.99272209e-02,  8.44673887e-02, -2.77793556e-02, ...,
          -2.29256526e-02, -8.92228931e-02, -1.43634677e-02],
         [ 1.63772330e-02, -4.10966054e-02,  8.70788023e-02, ...,
          -5.99436872e-02, -8.68537426e-02,  8.15365687e-02]],

        [[-2.24211887e-02,  1.03235804e-01, -5.92574738e-02, ...,
           1.00236334e-01,  5.13677374e-02, -2.97042280e-02],
         [ 1.26084387e-02, -7.60462582e-02, -1.46412775e-02, ...,
           

ValueError: You called `set_weights(weights)` on layer "conv2d_21" with a weight list of length 0, but the layer was expecting 2 weights. Provided weights: []...